<a href="https://colab.research.google.com/github/fwbane/nlp/blob/master/Simple_English_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
import pandas as pd

In [7]:
from google.colab import files
uploaded = files.upload()

Saving simplewiki-20191101-pages-meta-current.xml.bz2 to simplewiki-20191101-pages-meta-current.xml.bz2


In [0]:
# def load_simple_english():
#   df = pd.read_csv('simplewiki-20191101-pages-articles-multistream.csv', quotechar='|', index_col = False)
#   df['timestamp'] = pd.to_datetime(df['timestamp'],format='%Y-%m-%dT%H:%M:%SZ')
#   return df

In [5]:
# corpus = load_simple_english()
# corpus.head()

,page_id,page_title,page_ns,revision_id,timestamp,contributor_id,contributor_name,bytes
0,1,April,0,6516364,2019-04-26 03:40:24,16695,ChenzwBot,-1
1,2,August,0,6516375,2019-04-26 03:43:59,16695,ChenzwBot,-1
2,6,Art,0,6635144,2019-08-14 02:31:34,756599,DannyS712,-1
3,8,A,0,6662174,2019-09-11 22:33:05,873501,Wyatt2049,-1
4,9,Air,0,6703509,2019-10-29 14:05:03,8630,Chenzw,-1


In [0]:
import bz2
import subprocess

data_path = 'simplewiki-20191101-pages-meta-current.xml.bz2'

In [9]:
%%timeit -n 3 -r 3

lines = []
for i, line in enumerate(bz2.BZ2File(data_path, 'r')):
    lines.append(line)
    if i > 1e6:
        break

3 loops, best of 3: 4.94 s per loop


In [10]:
%%timeit -n 3 -r 3

lines = []
for i, line in enumerate(subprocess.Popen(['bzcat'], 
                         stdin = open(data_path), 
                         stdout = subprocess.PIPE).stdout):
    lines.append(line)
    if i > 1e6:
        break

3 loops, best of 3: 3.2 s per loop


In [0]:
lines = []

for i, line in enumerate(subprocess.Popen(['bzcat'], 
                         stdin = open(data_path), 
                         stdout = subprocess.PIPE).stdout):
    lines.append(line)
    if i > 5e5:
        break
lines[-109:-1]

In [0]:
import xml.sax

class WikiXmlHandler(xml.sax.handler.ContentHandler):
    """Content handler for Wiki XML data using SAX"""
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._current_tag = None
        self._pages = []

    def characters(self, content):
        """Characters between opening and closing tags"""
        if self._current_tag:
            self._buffer.append(content)

    def startElement(self, name, attrs):
        """Opening tag of element"""
        if name in ('title', 'text', 'timestamp'):
            self._current_tag = name
            self._buffer = []

    def endElement(self, name):
        """Closing tag of element"""
        if name == self._current_tag:
            self._values[name] = ' '.join(self._buffer)

        if name == 'page':
            self._pages.append((self._values['title'], self._values['text']))

In [38]:
# Content handler for Wiki XML
handler = WikiXmlHandler()

# Parsing object
parser = xml.sax.make_parser()
parser.setContentHandler(handler)

handler._pages

[]

In [52]:
for l in lines[-169:]:
  print(l)
  l= l.decode('utf-8')
  print(l)
  parser.feed(l)

b'  <page>\n'
  <page>



SAXParseException: ignored

In [53]:
handler._pages

[('User:Tuohirulla',
  'Tuohirulla is a boy from [[Finland]]. Tuohirulla is at [[university]].')]

In [45]:
for line in subprocess.Popen(['bzcat'], 
                              stdin = open(data_path), 
                              stdout = subprocess.PIPE).stdout:
    parser.feed(line.decode('utf-8'))
    
    # Stop when 3 articles have been found
    if len(handler._pages) > 2:
        break

SAXParseException: ignored

In [49]:
line.decode('utf-8')

'<mediawiki xmlns="http://www.mediawiki.org/xml/export-0.10/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.mediawiki.org/xml/export-0.10/ http://www.mediawiki.org/xml/export-0.10.xsd" version="0.10" xml:lang="en">\n'